In [81]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [82]:
import numpy as np 
import pandas as pd
import seaborn as sns
import argparse
import sklearn.utils.random as sk_random

## Introducción al dataset

El dataset contiene 5531451 filas, cada una correspondiente a la información de un cliente de la empresa American Express con 190 columnas que brindan información de los clientes. Por motivos de privacidad el significado de los valores de dichas columnas no es brindado por la empresa. Sin embargo en lugar del significado de cada columna se nos proporciona las siguientes categorías según el nombre de las mismas:
- D_* = Variables relacionadas a la delincuencia
- S_* = Variables relacionadas a los gastos
- P_* = Variables relacionadas a los pagos
- B_* = Variables de balances
- R_* = Variables de riesgo

Por último se aclara que todas las variables numéricas están normalizadas y que las siguientes variables son categóricas:

- ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']


## Reducción del dataset

Debido a la gran cantidad de información del dataset y por lo tanto peso del mismo, se redujo la cantidad de filas en un 5%. De esta forma y redondeando el resultado se plantea trabajar con un dataset que contenga unas 276572 filas.
El procedimiento utilizado para la reducción del dataset es el provisto por la función sample_without_replacement de la libreria scikit-learn. Como parámetros de la misma se utilizó:
- La cantidad de filas totales del dataset(n_population) = 5531451
- La cantidad de filas que se desean obtener(n_samples) = 276572
- Valor semilla (random_state) = 576

*Aclaración: Este último valor se calculó como el resto obtenido al realizar la siguiente división con la multiplicación de una constante por el número del grupo (11) como numerador y a 1000 como denominador $\frac{31416 * 11}{1000}$*

## a) Visualización de los datos:

In [83]:
df_reducido = pd.read_parquet('/kaggle/input/train-data/result_2')

In [84]:
df_reducido.shape

In [85]:
df_reducido

### Tipos de variables

In [135]:
df_reducido.dtypes.value_counts()

Analizamos el tipo de las variables para conocer más de las mismas y determinar si serán de utilidad, o en su defecto si se debe realizar algún procedimiento para su modificación o eliminación.
Sabemos que las variables normalizadas pertenecen a la categoría “float64” y resulta de interés el análisis de las variables de tipo “object” u objetos y las variables “int” o enteras. 
- Las variables de tipo objeto son las columnas “customer_ID”,”S_2”,”D_63” y “D_64”
- Las variables de tipo entero son las columnas “target” y “B_31”

Se mostraran sus valores y se decidirá en base a los mismos que acciones tomar al momento de modificar el dataset.

In [158]:
df_reducido.loc[:,'S_2'] 

In [160]:
df_reducido.loc[:,'B_31']

In [161]:
df_reducido.loc[:,'D_63']

In [162]:
df_reducido.loc[:,'D_64']

### Categorias de las variables

In [99]:
columns = df_reducido.columns

sum_types = [0,0,0,0,0] # [type_D ,type_S,type_P,type_B, type_R ]

for column in columns:
    if (column[0] == 'D'):
        sum_types[0] += 1
    elif (column[0] == 'S'):
        sum_types[1] += 1
    elif (column[0] == 'P'):
        sum_types[2] += 1
    elif (column[0] == 'B'):
        sum_types[3] += 1
    elif (column[0] == 'R'):
        sum_types[4] += 1
sum_types

Observamos que en nuestro dataset reducido tenemos cantidades significativamente diferentes de cada tipo de variable, siendo 96 variables relacionadas a la delincuencia, 22 relacionadas a los gastos, 3 relacionadas a los pagos, 40 de balances y 28 variables de riesgo. Sumando un total 189 columnas donde se excluyen a dos variables ('customer_ID' y 'target') para completar las 191 variables tal y como se mostró anteriormente al realizar shape sobre el dataset.

In [88]:
df_reducido.sort_index(axis = 1, inplace = True)
df_reducido

### Correlación entre variables

In [103]:
sns.heatmap(df_reducido.corr(method='pearson'), cmap='Reds')

Del heatmap obtenido se pueden observar ciertas correlaciones, siendo muchas de estas con un color intenso (indicando un gran nivel de correlación) con variables de su mismo “tipo” según las categorías presentadas al inicio del trabajo (D, S, P, B, R). Esto nos indica que existe la posibilidad de realizar una reducción de la dimensionalidad de los datos sin perder información, lo cual será profundizado más adelante en el trabajo.
Paralelamente observamos que existe una “Cruz” creada por una línea vertical y una horizontal de color blanco indicando que la variable D_87 no posee ninguna correlación con ninguna otra variable. Esta situación es causada por la cantidad de datos Nan que posee esta columna (mostrado a continuación), más adelante en el trabajo se trabajara sobre los datos faltantes y se tomaran acciones sobre los mismos.


In [120]:
df_reducido.loc[:,'D_87']